In [1]:
import pandas as pd 

In [2]:
df = pd.read_json("all.json")

In [4]:
df = df.drop(columns=["depth", "parent", "exist_children", "product_count", "order", "status", "created_at", "updated_at"])

In [9]:
df = df.drop("slug", axis=1)

In [12]:
for i in range(len(df) // 100 + 1):
    df.iloc[i: i + 100].to_csv(f"{(i + 1)  * 100}.csv", index=False)
   

In [23]:
df.head()

,id,name
0,5371,Quloqchinlar va audio texnikalar
1,5400,Quloqchinlar
2,5243,Elektronikaa
3,5499,Smartfonlar
4,6396,Aksessuarlar


In [24]:
from google.cloud import translate_v3
PROJECT_ID = "phon-462405"
def translate_text(
    text: str = "YOUR_TEXT_TO_TRANSLATE",
    source_language_code: str = "uz",
    target_language_code: str = "en-US",
) -> translate_v3.TranslationServiceClient:
    """Translate Text from a Source language to a Target language.
    Args:
        text: The content to translate.
        source_language_code: The code of the source language.
        target_language_code: The code of the target language.
            For example: "fr" for French, "es" for Spanish, etc.
            Find available languages and codes here:
            https://cloud.google.com/translate/docs/languages#neural_machine_translation_model
    """

    # Initialize Translation client.
    client = translate_v3.TranslationServiceClient()
    parent = f"projects/{PROJECT_ID}/locations/global"

    # MIME type of the content to translate.
    # Supported MIME types:
    # https://cloud.google.com/translate/docs/supported-formats
    mime_type = "text/plain"

    # Translate text from the source to the target language.
    response = client.translate_text(
        contents=[text],
        parent=parent,
        mime_type=mime_type,
        source_language_code=source_language_code,
        target_language_code=target_language_code,
    )

    # Display the translation for the text.
    # For example, for "Hello! How are you doing today?":
    # Translated text: Bonjour comment vas-tu aujourd'hui?
    for translation in response.translations:
        print(f"Translated text: {translation.translated_text}")

    return response



def translate(row):
    return translate_text(row["name"], "uz", "en-US")


In [30]:
from concurrent.futures import ThreadPoolExecutor

def translate_row(row):
    return translate_text(row["name"], "uz", "ru")

with ThreadPoolExecutor(max_workers=20) as executor:
    translations = list(executor.map(translate_row, [row for _, row in df.iterrows()]))

df["name_ru"] = translations

Translated text: Бытовая техника
Translated text: Стиральные машины
Translated text: Дорожные аксессуары
Translated text: Аксессуары для электроники
Translated text: Наушники
Translated text: Товары для сна
Translated text: Климатическая инженерия
Translated text: Смартфоны
Translated text: Аксессуары
Translated text: Аптека
Translated text: Крупная бытовая техника
Translated text: Электроника
Translated text: Предметы домашнего обихода
Translated text: Дорожные сумки
Translated text: Фанаты
Translated text: Смартфоны и телефоны
Translated text: Обувь
Translated text: Предметы домашнего обихода
Translated text: Наушники и аудиооборудование
Translated text: Компьютерные технологии
Translated text: Кнопочные телефоны
Translated text: Тойота
Translated text: Женская обувь
Translated text: Аккумуляторные батареи
Translated text: Телевизоры и видеотехника
Translated text: Женские аксессуары
Translated text: Аксессуары для бассейна
Translated text: Машины
Translated text: Ноутбуки
Translated

In [54]:

df = pd.read_csv("all.csv")

In [52]:
df

,id,name_en,name_ru,name_uz
0,5371,Headphones and audio equipment,Наушники и аудиооборудование,Quloqchinlar va audio texnikalar
1,5400,Headphones,Наушники,Quloqchinlar
2,5243,Electronics,Электроника,Elektronikaa
3,5499,Smartphones,Смартфоны,Smartfonlar
4,6396,Accessories,Аксессуары,Aksessuarlar
...,...,...,...,...
1586,5257,Cable protection,Защита кабеля,Kabel himoyasi
1587,5263,Voltage converters,Преобразователи напряжения,Kuchlanishni oʻzgartirgichlar
1588,5267,Electrical outlets,Электрические розетки,Elektr tarmoqlagichlar
1589,5255,Battery chargers,Зарядные устройства для аккумуляторов,Akkumulyatorlar uchun zaryadlovchi qurilmalari


In [55]:
classes_map = {x["id"]: {"name_uz":  x["name_uz"], "name_ru": x["name_ru"], "name_en": x["name_en"]} for x in df.to_dict(orient="records")}

In [56]:
classes_map

{5371: {'name_uz': 'Quloqchinlar va audio texnikalar',
  'name_ru': 'Наушники и аудиооборудование',
  'name_en': 'Headphones and audio equipment'},
 5400: {'name_uz': 'Quloqchinlar',
  'name_ru': 'Наушники',
  'name_en': 'Headphones'},
 5243: {'name_uz': 'Elektronikaa',
  'name_ru': 'Электроника',
  'name_en': 'Electronics'},
 5499: {'name_uz': 'Smartfonlar',
  'name_ru': 'Смартфоны',
  'name_en': 'Smartphones'},
 6396: {'name_uz': 'Aksessuarlar',
  'name_ru': 'Аксессуары',
  'name_en': 'Accessories'},
 7378: {'name_uz': 'Xoʻjalik buyumlari',
  'name_ru': 'Предметы домашнего обихода',
  'name_en': 'Household items'},
 5473: {'name_uz': 'Smartfonlar va telefonlar',
  'name_ru': 'Смартфоны и телефоны',
  'name_en': 'Smartphones and phones'},
 5657: {'name_uz': 'Katta maishiy texnika',
  'name_ru': 'Крупная бытовая техника',
  'name_en': 'Large household appliances'},
 7142: {'name_uz': 'Uy-roʻzgʻor buyumlari',
  'name_ru': 'Предметы домашнего обихода',
  'name_en': 'Household items'},
 6

In [57]:
df = pd.read_csv("all.csv")
classes_map = {x["id"]: {"name_uz":  x["name_uz"], "name_ru": x["name_ru"], "name_en": x["name_en"]} for x in df.to_dict(orient="records")}

In [58]:
classes_map

{5371: {'name_uz': 'Quloqchinlar va audio texnikalar',
  'name_ru': 'Наушники и аудиооборудование',
  'name_en': 'Headphones and audio equipment'},
 5400: {'name_uz': 'Quloqchinlar',
  'name_ru': 'Наушники',
  'name_en': 'Headphones'},
 5243: {'name_uz': 'Elektronikaa',
  'name_ru': 'Электроника',
  'name_en': 'Electronics'},
 5499: {'name_uz': 'Smartfonlar',
  'name_ru': 'Смартфоны',
  'name_en': 'Smartphones'},
 6396: {'name_uz': 'Aksessuarlar',
  'name_ru': 'Аксессуары',
  'name_en': 'Accessories'},
 7378: {'name_uz': 'Xoʻjalik buyumlari',
  'name_ru': 'Предметы домашнего обихода',
  'name_en': 'Household items'},
 5473: {'name_uz': 'Smartfonlar va telefonlar',
  'name_ru': 'Смартфоны и телефоны',
  'name_en': 'Smartphones and phones'},
 5657: {'name_uz': 'Katta maishiy texnika',
  'name_ru': 'Крупная бытовая техника',
  'name_en': 'Large household appliances'},
 7142: {'name_uz': 'Uy-roʻzgʻor buyumlari',
  'name_ru': 'Предметы домашнего обихода',
  'name_en': 'Household items'},
 6